In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.applications import ResNet50, ResNet152, InceptionV3, InceptionResNetV2, Xception, DenseNet121, DenseNet169, DenseNet201
from tensorflow.keras.applications.resnet import preprocess_input as resnet_preprocess_input
from tensorflow.keras.applications.inception_v3 import preprocess_input as inception_preprocess_input
from tensorflow.keras.applications.xception import preprocess_input as xception_preprocess_input
from tensorflow.keras.applications.densenet import preprocess_input as densenet_preprocess_input
from tensorflow.keras.callbacks import LearningRateScheduler
import math

# Set random seed for reproducibility
SEED = 42
tf.random.set_seed(SEED)

# Check GPU availability
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Directories for training and testing data
train_dir = "/kaggle/input/skin-lesion-dataset-new/Skin Lesion Input Dataset New/Train_Set_Images"
test_dir = "/kaggle/input/skin-lesion-dataset-new/Skin Lesion Input Dataset New/Test_Set_Images"

IMG_SIZE = (224, 224)
BATCH_SIZE = 64

# Data generators
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    seed=SEED
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Function to build models dynamically
def build_model(model_name):
    if model_name == 'ResNet50':
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        preprocess_func = resnet_preprocess_input
    elif model_name == 'ResNet152':
        base_model = ResNet152(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        preprocess_func = resnet_preprocess_input
    elif model_name == 'InceptionV3':
        base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        preprocess_func = inception_preprocess_input
    elif model_name == 'InceptionResNetV2':
        base_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        preprocess_func = inception_preprocess_input
    elif model_name == 'Xception':
        base_model = Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        preprocess_func = xception_preprocess_input
    elif model_name == 'DenseNet121':
        base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        preprocess_func = densenet_preprocess_input
    elif model_name == 'DenseNet169':
        base_model = DenseNet169(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        preprocess_func = densenet_preprocess_input
    elif model_name == 'DenseNet201':
        base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        preprocess_func = densenet_preprocess_input

    
    # Build the model
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(128, activation='relu'),
        Dropout(0.2),  # Set dropout rate to 0.2
        Dense(train_generator.num_classes, activation='softmax')
    ])
    return model

# List of models to train
models_to_train = ['Xception']

# Training loop
for model_name in models_to_train:
    print(f"\nTraining {model_name}...\n")
    model = build_model(model_name)

    # Compile the model
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    # Learning Rate Scheduler
    def step_decay(epoch):
        initial_lr = 0.001
        drop_rate = 0.1
        epochs_drop = 5
        new_lr = initial_lr * math.pow(drop_rate, math.floor((1 + epoch) / epochs_drop))
        return new_lr
    
    lr_scheduler = LearningRateScheduler(step_decay)
    
    # Train the model
    history = model.fit(
        train_generator,
        epochs=20,
        validation_data=test_generator,
        callbacks=[lr_scheduler],
        verbose=1
    )

    # Evaluate the model
    train_loss, train_acc = model.evaluate(train_generator)
    test_loss, test_acc = model.evaluate(test_generator)

    print(f"{model_name} Training Accuracy: {train_acc*100:.2f}%")
    print(f"{model_name} Testing Accuracy: {test_acc*100:.2f}%")

Num GPUs Available:  2
Found 11438 images belonging to 8 classes.
Found 2864 images belonging to 8 classes.

Training Xception...

83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/20


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1734348377.109454     103 service.cc:145] XLA service 0x7d3fa002f070 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1734348377.109508     103 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1734348377.109512     103 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1734348428.907725     103 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


179/179 ━━━━━━━━━━━━━━━━━━━━ 323s 1s/step - accuracy: 0.5414 - loss: 1.1993 - val_accuracy: 0.3792 - val_loss: 3.2116 - learning_rate: 0.0010
Epoch 2/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 188s 1s/step - accuracy: 0.7602 - loss: 0.6406 - val_accuracy: 0.6215 - val_loss: 1.1878 - learning_rate: 0.0010
Epoch 3/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 187s 1s/step - accuracy: 0.8234 - loss: 0.4609 - val_accuracy: 0.5964 - val_loss: 1.1476 - learning_rate: 0.0010
Epoch 4/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 187s 1s/step - accuracy: 0.8754 - loss: 0.3502 - val_accuracy: 0.6739 - val_loss: 1.0839 - learning_rate: 0.0010
Epoch 5/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 186s 1s/step - accuracy: 0.9331 - loss: 0.1933 - val_accuracy: 0.8369 - val_loss: 0.5177 - learning_rate: 1.0000e-04
Epoch 6/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 186s 1s/step - accuracy: 0.9808 - loss: 0.0690 - val_accuracy: 0.8439 - val_loss: 0.5578 - learning_rate: 1.0000e-04
Epoch 7/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 187s 1s/step - accuracy: 0.9882 - loss: 0.036